# Scenario 2: A cross-functional team with one data scientist working on an ML model

Scenerio setup: 
+ Tracking server: yes, local server.
+ Backend store: sqlite database.
+ Artifact store: local filesystem.

The experiment can be explored locally by accessing the local tracking server.<br>
To run this example, we need to launch the mlflow server locally by this command: 


`
mlflow server --backend-store-uri sqlite://backend.db --default-artifact-root ./artifacts_local
`

In [63]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000/")

In [64]:
print(f"Mlflow tracking URI: {mlflow.get_tracking_uri()}")

Mlflow tracking URI: http://127.0.0.1:5000/


In [65]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/artifacts_local/0', creation_time=1717845335581, experiment_id='0', last_update_time=1717845335581, lifecycle_stage='active', name='Default', tags={}>]

## Create an experiment and log a new run

In [67]:
mlflow.end_run()

RestException: RESOURCE_DOES_NOT_EXIST: Run with id=42cfa39879d0494c9eb514fccf900ee3 not found

In [68]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():
    X, y = load_iris(return_X_y=True)
    
    params = { "C": 0.1, "random_state": 42 }
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params)
    lr = lr.fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifact URI: {mlflow.get_artifact_uri()}") 

default artifact URI: file:///D:/code/nda-mlops-zoomcamp-2024/week-02/artifacts_local/1/d5c99529b7214afb82191601d37db10d/artifacts


In [69]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/artifacts_local/1', creation_time=1717845580370, experiment_id='1', last_update_time=1717845580370, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/artifacts_local/0', creation_time=1717845335581, experiment_id='0', last_update_time=1717845335581, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the Model Registry

In [74]:
from mlflow.client import MlflowClient

client = MlflowClient("http://127.0.0.1:5000/")

In [75]:
print(client.search_registered_models())

[]


In [82]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id

In [84]:
mlflow.register_model(
    model_uri=f"run:/{run_id}/models",
    name="iris_classifier"
)

Successfully registered model 'iris_classifier'.
2024/06/08 18:26:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_classifier, version 1
Created version '1' of model 'iris_classifier'.


<ModelVersion: aliases=[], creation_timestamp=1717846003303, current_stage='None', description='', last_updated_timestamp=1717846003303, name='iris_classifier', run_id='', run_link='', source='run:/d5c99529b7214afb82191601d37db10d/models', status='READY', status_message='', tags={}, user_id='', version='1'>